In [ ]:
import os
import requests
from urllib3.exceptions import NewConnectionError
import uuid
from bs4 import BeautifulSoup

In [ ]:
classes = ['Coniferous', 'Deciduous']
page_length = 10

In [ ]:
def get_image_links(query, nth_page):
    """Retrieve the image links from the response of the ecosia request."""
    image_links = []
    
    # Add headers to prevent IP block
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
        'content-type': 'text/html; charset=utf-8',
    }
    params = {
        'q': query,
        'p': nth_page
    }
    response = requests.get('https://www.ecosia.org/images', headers=headers, params=params)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    image_results = soup.select('a.image-result')
    for img in image_results:
        image_links.append(img.get('href'))
    return image_links

In [ ]:
def download_images(img_links, folder_name, name_prefix):
    """Download images from the links and save them to a folder with name prefix."""
    for link in img_links:
        try:
            img = requests.get(link)
            # Image name = name_prefix + _uuid4 + .jpg
            path = f'./{folder_name}/{name_prefix}_{uuid.uuid4().hex}.jpg'
            with open(path, 'wb') as handle:
                handle.write(img.content)
        except:
            print(f'Unable to download image from the link: {link}')

In [ ]:
for class_ in classes:
    try:
        # Folder name = class name + _images
        folder_name = f'{class_}_images'
        os.mkdir(folder_name)
    except FileExistsError:
        pass
    for i in range(1, page_length + 1):
        img_links = get_image_links(query=class_, nth_page=i)
        download_images(img_links, folder_name, name_prefix=class_)